In [1]:
from train_tf2 import Trainer
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf

train_filenames = "../data/training_flat_156_3d.tfrecords"
val_filenames = "../data/validation_flat_156_3d.tfrecords"
#Trainer(model='FF').run(train_filenames, val_filenames, x_slice=127)
Trainer(model='CNN').run(train_filenames, val_filenames, x_slice=127)

Epoch  #1 |                   |   0% Cost: 5339322.5000, Avg. Acc: 0.4000, Current Acc: 0.50, elapsed: 16s, current: 1.4151

ResourceExhaustedError: Failed to allocate memory for the batch of component 0 [Op:IteratorGetNextSync]

In [ ]:
def slice_accuracy(dim="x"):
    accuracies = []
    range_ = np.arange(50, 206, 15)
    for slice_ in range_:
        if dim == "x":
            accuracies.append(Trainer(model='CNN').run(train_filenames, val_filenames, x_slice=slice_))
        elif dim == "y":
            accuracies.append(Trainer(model='CNN').run(train_filenames, val_filenames, y_slice=slice_))
        elif dim == "z":
            accuracies.append(Trainer(model='CNN').run(train_filenames, val_filenames, z_slice=slice_))
    return accuracies
    
accuracies = slice_accuracy(dim="x")    
plt.plot(range_, accuracies)
plt.title("Accuracies for 2d slices in x-dimension")
plt.xlabel("pixel")
plt.ylabel("accuracy")
plt.show()

Epoch  #1 |################## |  93% Cost 1.6135, Avg. Accuracy 0.2047, Current Accuracy: 0.20

In [3]:
import tensorflow as tf
name = "../data/training_flat_256_3d.tfrecords"
def _parse_function(example_proto):
    features = {"image": tf.io.FixedLenFeature([256, 256, 256], tf.float32),
              "label": tf.io.FixedLenFeature((), tf.int64),
               'name': tf.io.FixedLenFeature((), tf.string, default_value='')}
    parsed_features = tf.io.parse_single_example(serialized=example_proto, features=features)
    parsed_features["image"]= tf.reshape(parsed_features["image"],[-1]) 
    parsed_features["label"]=tf.one_hot(parsed_features["label"],5)
    return parsed_features["image"], parsed_features["label"], parsed_features["name"]

    return parsed_features["image"]
batchsize = 1
dataset = tf.data.TFRecordDataset(name).map(_parse_function)#.shuffle(
                    #buffer_size=10).batch(batchsize)
count=0
#for image, label, name in dataset:
#    count +=1
    #im_ = tf.reshape(image, [batchsize, 256, 256, 256])
    #label = tf.reshape(label, [batchsize, 5])
    #name = tf.reshape(name, [batchsize, -1])
    
    #print(im_[1, :, :, :].shape)
    #print(label)
#    print("\r" + str(count), end="")
# Count the records
#records_n = sum(1 for record in dataset)

#print("records_n = {}".format(records_n))

In [56]:
class model():
    
    def cost(self, X, y):
    #activations, logits = self.model(X, y, batch_size)
        return tf.reduce_mean(input_tensor=tf.nn.softmax_cross_entropy_with_logits(logits=X, labels=y))

    def forward(self, X):
        self.w_fc = tf.Variable(tf.random.truncated_normal(shape=[100, 5], dtype=tf.float64, stddev=0.1), name='w_{0}'.format("hi"))
        self.b_fc = tf.Variable(tf.constant(0.0, shape=[5], dtype=tf.float64), name='b_{0}'.format("hi"))
        self.variables = [self.w_fc, self.b_fc]
        return tf.nn.relu(tf.nn.bias_add(tf.matmul(X, self.w_fc), self.b_fc), name="hi")
    
    def backward(self, X_train, y_train):
        optimizer = tf.compat.v1.train.AdamOptimizer(0.00001)
        with tf.GradientTape() as tape:
            predicted = self.forward(X_train)
            current_loss = self.cost(predicted, y_train)
        grads = tape.gradient(current_loss, self.variables)
        optimizer.apply_gradients(zip(grads, self.variables))



In [3]:
import tensorflow as tf
import numpy as np
#optimizer = tf.compat.v1.train.AdamOptimizer(0.00001)
x = np.random.rand(5, 100)
y = (np.sum(x, axis=1)/10).astype(int)

#w_fc = tf.Variable(tf.random.truncated_normal(shape=[100, 5], dtype=tf.float64, stddev=0.1), name='w_{0}'.format("hi"))
#tf.matmul(x, w_fc)
model_default = model()

preds = model_default.forward(x)

#loss_total = loss_total + model_default.loss(preds, y)

model_default.backward(x, y)

NameError: name 'model' is not defined

In [3]:
import tensorflow as tf
from model_tf2 import MRI, MRI_2dCNN
from utils_tf2 import ModelSelectionError
import time
logdir = 'logs_x_FFNN/'
chkpt = 'logs_x_FFNN/model.ckpt'
n_epochs = 10 
batch_size = 10
label_size = 3

class Trainer:    
    mirrored_strategy = tf.distribute.MirroredStrategy()
    with mirrored_strategy.scope() as strategy:
        def __init__(self, model='FF', im_size=156):
            try:
                if model =='FF':
                    self.model = MRI(train=True, im_size=im_size, label_size=label_size)
                elif model=='CNN':
                     self.model = MRI_2dCNN(train=True, im_size=im_size, label_size=label_size)
                elif (model != 'FF') | (model != 'CNN'):
                    raise ModelSelectionError()
            except ModelSelectionError:
                 print("Please select either FF for a Fully connected Feed-Forward network or CNN for a convolutional network.")
                    
       #@tf.function            
       #def cost(self, y_pred, y_true):
       #    return tf.reduce_mean(input_tensor=tf.nn.softmax_cross_entropy_with_logits(logits=y_pred, labels=y_true))

       #@tf.function 
       #def accuracy(self, y_pred, y_true):
       #    equals = tf.equal(tf.argmax(input=y_pred, axis=1), tf.argmax(input=y_true, axis=1))
       #    return tf.reduce_mean(input_tensor=tf.cast(equals, tf.float32))
       #
        def run(self, train_filenames, val_filenames, x_slice=None, y_slice=None, z_slice=None, strategy=strategy):
                self.checkpoint = tf.train.Checkpoint(optimizer=self.model.optimizer) 
                self.file_writer = tf.summary.create_file_writer(logdir)
                self.dataset = tf.data.TFRecordDataset(train_filenames).map(_parse_function).shuffle(
                    buffer_size=20).batch(batch_size)#.repeat()
                self.dist_dataset = strategy.experimental_distribute_dataset(self.dataset)
                self.val_dataset = tf.data.TFRecordDataset(val_filenames).map(_parse_function).shuffle(
                    buffer_size=20).batch(batch_size)#.repeat()
                self.dist_val_dataset = strategy.experimental_distribute_dataset(self.val_dataset)

                for epoch in range(n_epochs):
                    t_start = time.time()
                    self.train(epoch, x_slice, y_slice, z_slice, t_start)
                    final_accuracy = self.validate(epoch, x_slice, y_slice, z_slice)
                    self.checkpoint.save(chkpt)
                
                return final_accuracy
def _parse_function(example_proto):
            features = {"image": tf.io.FixedLenFeature([256, 256, 256], tf.float32),
                      "label": tf.io.FixedLenFeature((), tf.int64),
                       'name': tf.io.FixedLenFeature((), tf.string, default_value='')}
            parsed_features = tf.io.parse_single_example(serialized=example_proto, features=features)
            parsed_features["image"] = tf.reshape(parsed_features["image"], [-1]) 
            parsed_features["label"] = tf.one_hot(parsed_features["label"], label_size)
            return parsed_features["image"], parsed_features["label"], parsed_features["name"]

Trainer(model='CNN').run(train_filenames, val_filenames, x_slice=127)

AttributeError: 'Trainer' object has no attribute 'train'

In [9]:
tensor = tf.constant([[[1,2,3],[5,6,7]],[[1,5,10],[10,20,30]]])
np.array(tf.shape(tensor))
im_size_squared = 16
shape2 = tf.constant([3, 3, 32, 64])
shape_flat = tf.constant(int(shape2[-1] * im_size_squared / 4))
if tf.shape(tf.shape(tensor)) == 3 :
    print("HEY")
a = tf.constant([int(tf.constant(389376, dtype=tf.int32)), 10])

HEY


In [6]:
import tensorflow as tf
strategy = tf.distribute.MirroredStrategy()
with strategy.scope() as strategy:
    @tf.function
    def test_():
        print(tf.executing_eagerly())
        avg_cost = tf.constant(10, dtype=tf.float32)
        batch = 15.0
        d = 5
        f = d*avg_cost
        avg_hi = (avg_cost / (batch))
        print(avg_hi)
        return f
test_()

False
Tensor("truediv:0", shape=(), dtype=float32)


<tf.Tensor: id=39, shape=(), dtype=float32, numpy=50.0>

In [4]:
import tensorflow as tf
import time
strategy = tf.distribute.MirroredStrategy()
t_start = time.time()
t0 = time.time()
with strategy.scope() as strategy:
    @tf.function
    def test():
        n_batches = 1290#tf.constant(1290, dtype=tf.float32)
        batch = 150#tf.constant(15, dtype=tf.float32)
        batch += 1
        avg_cost = 0.0
        avg_accuracy = 0.0
        #batch = 10
        #n_batches = 100
        completion = batch / n_batches
        #print(completion.dtype)
        print_str = '|' + int(completion * 20) * '#' + (19 - int(completion * 20))  * ' ' + '|'
        t1 = time.time()
        batch_time = t1-t0
        epoch_time = t1 - t_start
        epoch = 1
        #avg_cost = 0.2324
        #batch = 10
        #avg_accuracy = 4.323
        batch_accuracy = 2.323
        print((avg_cost / (batch)))
        print(('\rEpoch {0:>3} {1} {2:3.0f}% Cost: {3:6.4f}, Avg. Accuracy: {4:6.4f}, Current Accuracy: {5:.2f}, ' +
                      'elapsed time: {6:.0f}s').format(
                      '#' + str(epoch + 1), print_str, completion * 100, (avg_cost / (batch)),(avg_accuracy / (batch)), 
                      batch_accuracy, epoch_time), end='')
        
        print(('\rEpoch {0:>3} {1} {2:3.0f}% Cost: {3:6.4f}, Avg. Accuracy: {4:6.4f}, Current Accuracy: {5:.2f}, ' +
                      'elapsed time: {6:.0f}s').format(
                      '#' + str(epoch + 1), print_str, completion * 100, avg_cost / (batch), avg_accuracy / (batch), 
                      batch_accuracy, epoch_time), end='')
        a = 'a' * int(3.5)
        print(a)
        return print_str
print(str(test().numpy(), 'utf-8'))

0.0
Epoch  #2 |##                 |  12% Cost: 0.0000, Avg. Accuracy: 0.0000, Current Accuracy: 2.32, elapsed time: 0saaa
|##                 |


In [3]:
print(1)

1


In [ ]:
print(3)

In [5]:
import tensorflow as tf
import time
t_start = time.time()
def print_(completion, epoch, avg_cost, batch, avg_accuracy, batch_accuracy, epoch_time):
    print(float(completion))
    print_str = '|' + int(completion * 20) * '#' + (19 - int(completion * 20))  * ' ' + '|'
    print(('\rEpoch {0:>3} {1} {2:3.0f}% Cost: {3:6.4f}, Avg. Accuracy: {4:6.4f}, Current Accuracy: {5:.2f}, ' +
                      'elapsed time: {6:.0f}s').format(
                      '#' + str(epoch + 1), print_str, completion * 100, avg_cost / (batch), avg_accuracy / (batch), 
                      batch_accuracy, epoch_time), end='')
strategy = tf.distribute.MirroredStrategy()
with strategy.scope() as strategy:
    @tf.function
    def test():
            epoch = 3
            n_batches = 1290
            batch = 0
            avg_cost = 0.0
            avg_accuracy = 0.0
            times = []
            for x_batch, y_batch, _name in dataset:
                t0 = time.time()
                for i in range(5):
                    batch += 1
                x_batch = tf.cast(x_batch, dtype=tf.float32)
                y_batch = tf.cast(y_batch, dtype=tf.float32)
                
                batch_accuracy = tf.constant(0.45, dtype=tf.float32)
                current_loss = tf.constant(143, dtype=tf.float32)
                avg_cost += current_loss
                avg_accuracy += batch_accuracy
                #self.file_writer.add_summary(summ, epoch * n_batches + batch)
                completion = batch / n_batches
                print(type(batch))
                print(type(completion))
                
                completion = tf.cast(completion, dtype=tf.float32)
                completion = completion.eval(session=tf.compat.v1.Session()) 
                print(type(completion), completion.dtype)
                t1 = time.time()
                batch_time = t1-t0
                epoch_time = t1 - t_start
                print_(completion, epoch, avg_cost, batch, avg_accuracy, batch_accuracy, epoch_time)

print(str(test().numpy(), 'utf-8'))

<class 'tensorflow.python.framework.ops.Tensor'>
<class 'tensorflow.python.framework.ops.Tensor'>


InvalidArgumentError: in converted code:

    <ipython-input-5-b57910c532fb>:21 test  *
        for x_batch, y_batch, _name in dataset:
    /usr/local/lib/python3.5/dist-packages/tensorflow_core/python/autograph/operators/control_flow.py:651 reduce_body
        new_vars = body(iterate, *loop_vars)
    /usr/local/lib/python3.5/dist-packages/tensorflow_core/python/framework/ops.py:798 eval
        return _eval_using_default_session(self, feed_dict, self.graph, session)
    /usr/local/lib/python3.5/dist-packages/tensorflow_core/python/framework/ops.py:5410 _eval_using_default_session
        return session.run(tensors, feed_dict)
    /usr/local/lib/python3.5/dist-packages/tensorflow_core/python/client/session.py:956 run
        run_metadata_ptr)
    /usr/local/lib/python3.5/dist-packages/tensorflow_core/python/client/session.py:1180 _run
        feed_dict_tensor, options, run_metadata)
    /usr/local/lib/python3.5/dist-packages/tensorflow_core/python/client/session.py:1359 _do_run
       

In [2]:
a = tf.constant(4)
a.dtype

tf.int32

In [1]:
3

3